In [ ]:
import os
import numpy as np
import pandas as pd
import osmnx as ox

print("osmnx version",ox.__version__)

In [ ]:
# read the train csv file
df = pd.read_csv("./data/trajectory/train.csv")

# only keep the first 1500 row
df_filtered = df.head(1500).copy()
df_filtered["POLYLINE"] = df_filtered["POLYLINE"].apply(eval)

# save the filtered data to csv file
df_filtered.to_json("./data/trajectory/train-1500.json", orient="records", indent=4)

In [ ]:
def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = np.arange(0, gdf_edges.shape[0], dtype='int')
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

In [ ]:
place_name = "Porto, Portugal"
network_type = "drive"
G = ox.graph_from_place(place_name, network_type=network_type)
save_graph_shapefile_directional(G, filepath='./data/map')
ox.save_graphml(G, "./data/map/porto.graphml")